In [86]:
import spacy
import json
from datetime import datetime, timedelta
import typing as _type
import re
nlp = spacy.load('en_core_web_sm')

quantity_pos = ["NUM"]
quantity_tag = ["CD"]
quantity_identifiers = ["¼", "½", "¾", "⅓", "⅔",
                        "⅕", "⅖", "⅗", "⅘", "⅙", "⅚", "⅛", "⅜", "⅝", "⅞"]
quantity_unit_identifiers = ["g", "gms", "grams", "l", "litres", "ml", "mL", "L", "dl", "dL", "teaspoon", "tablespoon", "cup", "pinch", "pinche", "piece", "pieces", "pinches", "handful",
                             "pint", "quart", "quarters", "gallon", "mg", "pound", "ounce", "mm", "cm", "mililitres", "kg", "kilograms", "tsp", "tbsp"]


In [91]:
strings = [
    'Whole wheat flour 3/4 cup',

    'Gram flour (besan) 1/4 cup',

    'Salt to taste',

    'Peanuts 1 1/2 teaspoons',

    'Turmeric powder 3/4 teaspoon',

    'Red chilli powder 3/4 teaspoon',

    'Asafoetida 2 pinches',

    'Oil 2 tablespoons',

    'Desi ghee 3 tablespoon',

    'Dried red chillies 2',

    'Fenugreek seeds (methi dana) 1/4 teaspoon',

    'Mustard seeds 3/4 teaspoon',

    'Curry leaves 10',

    'Jaggery (gur), grated 1 teaspoon',

    'Green chilli paste 1 1/2 teaspoon',

    'Onion , finely chopped 1 small',

    'Fresh coriander leaves, chopped 2 tablespoons',
]


phrases = [
    "I am going",
    "200g prawns",
    "200g crab meat",
    "1 large potato, cooked until just soft & grated",
    "4 curry leaves, chopped",
    "handful of fresh coriander leaves",
    "mustard oil for frying",
    "1 tsp turmeric",
    "1 fresh chilli, finely chopped",
    "½ tsp Kashmiri chilli powder",
    "1 tsp garam masala",
    "½ tsp salt or to taste",
    "juice from 1 lemon",
    "3 tbsp gram flour",
    "1 tsp chilli flakes",
]

phrases2=[
"1 tsp cumin seeds",
"1 bunch fresh coriander, leaves and stems",
"½ bunch fresh mint leaves,  ",
"4 garlic cloves",
"3cm piece of fresh ginger",
"4 fresh green chillies",
"1 tsp salt",
"½ lemon, juice only",
"1 tbsp oil",
"3 tbsp thick Greek yogurt",
]

phrases3=[
"1 small cauliflower",
"2 potatoes, peeled cut into 4cm cubes",
"2 tbsp mustard oil",
"1 tsp mustard seeds",
"1 tsp cumin seeds",
"1 onion, finely chopped",
"2 cloves garlic, finely chopped",
"200g/½ tin tomatoes",
"1 tbsp ginger, grated",
"1 tsp salt",
"1 tsp turmeric",
"1 chilli, finely chopped",
"1 tsp dried fenugreek",
"1 tsp garam masala",
"Handful chopped fresh coriander",
]

In [88]:
def get_quantity_identifier_regexp(quantity_identifiers):
    re_quantity_identifier_str = '\w*[' + \
        ''.join(quantity_identifiers) + ']\w*'
    quantity_identifier_regexp = '('+re_quantity_identifier_str+')'
    return quantity_identifier_regexp


def get_quantity_unit_identifier_regexp(quantity_unit_identifiers):
    re_quantity_unit_identifier_str = '(' + \
        '|'.join(quantity_unit_identifiers) + ')'
    quantity_unit_identifier_regexp = re_quantity_unit_identifier_str
    return quantity_unit_identifier_regexp


In [94]:
for phrase in phrases3:
    quantity_identifier_regexp = get_quantity_identifier_regexp(
        quantity_identifiers)
    quantity_unit_identifier_regexp = get_quantity_unit_identifier_regexp(
        quantity_unit_identifiers)
    name = []
    name_string = ""
    quantity = []
    quantity_unit = []
    ingredient = []
    prep_hint = ""
    phrase = " ".join(re.split('(\d+)',phrase))
    doc = nlp(phrase)
    # print("-"*50)
    # print(phrase)
    deleted_tokens = []
    indices = {c.strip().replace(",", ""): i for i,
               c in enumerate(phrase.split())}
    # print(indices)
    for token in doc:
        # print(token.text," ",token.pos_," ",token.dep_," ",token.tag_," ",token.is_stop)
        if((token.pos_ in quantity_pos) or (token.tag_ in quantity_tag) or (re.fullmatch(quantity_identifier_regexp, token.lemma_))):
            quantity.append(token.text)
            deleted_tokens.append(token.i)
        elif(re.fullmatch(quantity_unit_identifier_regexp, token.lemma_)):
            quantity_unit.append(token.text)
            deleted_tokens.append(token.i)
        elif (((token.tag_ in ['NN']) or (token.dep_ in ['nsubj', 'ROOT'])) and (token.pos_ in ['NOUN', 'PROPN'])):
            name.append(token.text)
            for child in token.children:
                # print(child.text," ",child.pos_," ",child.dep_)
                if((child.dep_ in ['amod', 'compound'])):
                    name.append(child.text)
                    deleted_tokens.append(child.i)
            # name.append(token.text)
            # print(name)
            name = [*set(name)]
            # print("name set: ",name)
            deleted_tokens.append(token.i)
    try:
        name_string = " ".join(sorted(name, key=indices.get))
    except:
        # print("indices exception")
        name_string = " ".join(name)
    for k in quantity_unit + quantity:
        # print k
        name_string = name_string.replace(k, "").strip()
    remaining_phrase = []
    for token in doc:
        if token.i not in deleted_tokens:
            remaining_phrase.append(token.text + " ")
    doc = nlp("".join(remaining_phrase))
    prep_hint = "".join(remaining_phrase)
    # print("Remaining Phrase: {}".format("".join(remaining_phrase)))
    print([name_string, " ".join(quantity),
          " ".join(quantity_unit), prep_hint])


--------------------------------------------------
 1  small cauliflower
['small cauliflower', '1', '', '    ']
--------------------------------------------------
 2  potatoes, peeled cut into  4 cm cubes
['potatoes peeled cut', '2 4', 'cm', '    , into   cubes ']
--------------------------------------------------
 2  tbsp mustard oil
['mustard oil', '2', 'tbsp', '    ']
--------------------------------------------------
 1  tsp mustard seeds
['mustard seeds', '1', 'tsp', '    ']
--------------------------------------------------
 1  tsp cumin seeds
['cumin seeds', '1', 'tsp', '    ']
--------------------------------------------------
 1  onion, finely chopped
['onion', '1', '', '    , finely chopped ']
--------------------------------------------------
 2  cloves garlic, finely chopped
['garlic', '2', '', '    cloves , finely chopped ']
--------------------------------------------------
 200 g/½ tin tomatoes
['g/½ tin tomatoes', '200', '', '  ']
---------------------------------------